### Visualization steps

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px 
from plotly.subplots import make_subplots

The 3 datasets are taken from kaggle and the author scraped it from Olympedia.org website: <br>
https://www.kaggle.com/datasets/josephcheng123456/olympic-historical-dataset-from-olympediaorg/data <br>

The data covers all Winter / Summer Olympic games from 1896 to 2022. <br>

"Olympic_Athlete_Event_Results.csv" Containing athlete_id, medal (Gold, Silver, Bronze or none) etc. <br>
"Olympic_Athlete_Bio.csv" Containing athlete bio like name, sex, height, weight. <br>
"Olympics_Country.csv" Containing country_noc (code) and its corresponding country name. <br>

There is some data discrepancy in these datasets. Maybe because the author scraped in in a wrong way or maybe because <br>
some data got changed over time. For example a Russian athlete won a medal but a year later the medal was removed, because <br>
this athlete was found guilty of an anti-doping violation. But overall, the data is extremely close to the real one. <br>

### My goals:

1. In my work I will examine how the number of athletes, participating countries and events changed over time. <br>
2. I will examine the difference between the Summer and Winter Olympics as well as difference in athletes' gender. <br>
3. I'd like to know how many athletes participate in more than one olympic event compared to those who participate in only one event. <br>
4. I will investigate how many women and woman's events percentage were in the first olympic games and in the last games. <br>
5. I'd like to know what age groups produce most of the medals and how it changed over time. <br>
6. Lastly I'd like to see top 20 countries by medals earned in 1988 and 1996. <br>
Many new countries emerged (fall of USSR), while Germany got reunited. How did that effect the number of medals earned? <br>

In [47]:
df = pd.read_csv("cleaned_olympic.csv")

In [48]:
gr1 = df.groupby(['year', 'season', 'sex'])['athlete_id'].nunique().reset_index().sort_values(by=['year']).rename({'athlete_id': 'cnt'}, axis=1)

In [49]:
fig = px.line(gr1, x='year', y='cnt', color='sex', facet_row='season', category_orders={"sex": ["Male", "Female"]},
              markers='o', height=650)
fig.add_scatter(x=[1932], y=[1300], row=2, col=1, text='US 1932', mode='text', showlegend=False)
fig.add_scatter(x=[1956], y=[2500], row=2, col=1, text='Australia 1956', mode='text', showlegend=False)
fig.add_scatter(x=[1980], y=[3600], row=2, col=1, text='USSR 1980', mode='text', showlegend=False) 

fig.update_traces(line_width=1.2, marker_size=4.5)
fig.update_layout(yaxis_title='Athletes', yaxis2_title='Athletes', xaxis_title='Year', legend = dict(x=0.995, y=0.5, 
                  font = dict(size = 13)), title='The total number of participants in Summer and Winter Olympics', title_x=0.5)
fig.update_xaxes(dtick=20)
fig.for_each_annotation(lambda a: a.update(x=1.001, font_size=13, textangle=0, text=f"<b>{a.text.split("=")[-1]}</b>"))
fig.show()

1. The poor participation in 1932 was the result of the worldwide economic decline (the Great Depression) and the expense of traveling to the US.

2. The remoteness of Australia and two international crises accounted for the low number of participants in 1956 Olympics. <br>
Four teams (Egypt, Iraq, Cambodia and Lebanon) boycotted in response to the Suez Crisis, in which Egypt was invaded by Israel, France and <br> 
the United Kingdom. Four teams (the Netherlands, Spain, Liechtenstein and Switzerland) boycotted in response to the Soviet invasion of Hungary.

1. The 1980 Summer Olympics boycott was one part of a number of actions initiated by the United States <br>
to protest against the Soviet invasion of Afghanistan. <br>



In [50]:
grouped = df.groupby(['year', 'season']).agg(country_cnt=('country', 'nunique'), event_cnt=('event', 'nunique')).reset_index()

In [51]:
fig1 = px.line(grouped, x='year', y='country_cnt', color='season', markers='o', 
               color_discrete_map = {'Winter': '#636efa', 'Summer': '#EF553B'}, height=400, width=800)
fig2 = px.line(grouped, x='year', y='event_cnt', color='season', markers='o', 
               color_discrete_map = {'Winter': '#636efa', 'Summer': '#EF553B'}, height=400, width=800)
fig1.update_layout(xaxis_title='Year', yaxis_title="") 
fig1.add_annotation(xref='paper', yref='paper', x=-0.135, y=0.55,
            text="<b>Countries</b>", showarrow=False, font_family="Open Sans", font_size=14, font_color='#2a3f5f')
fig2.update_layout(xaxis_title='Year', yaxis_title="") 
fig2.add_annotation(xref='paper', yref='paper', x=-0.13, y=0.55,
            text="<b>Events</b>", showarrow=False, font_family="Open Sans", font_size=14)
fig1.update_layout(title='Number of participating Countries over time', title_x=0.5, height=450)
fig2.update_layout(title='Number of participating Events over time', title_x=0.5)

fig1.add_annotation(x=1980, y=80, text='USSR', showarrow=True, ax=5, ay=20, yshift=-3)
fig1.add_annotation(x=1976, y=92, text='Canada', showarrow=True, ax=-20, ay=20)
fig1.show()
fig2.show()

1. We can see the significant drop again in 1980 Olympics that took place in Moscow, Soviet Union for the same reasons.
2. Also in 1976 in Canada Olympics 29 countries, mostly African, boycotted the Montreal Games when the International Olympic Committee (IOC) <br> refused to ban New Zealand, after the New Zealand national rugby union team had toured South Africa earlier in 1976 in defiance of the <br>United Nations' calls for a sporting embargo due to their racist apartheid policies. <br>
3. The number of events has been increasing, meaning that the Olympic sport becomes more diverse.

In [52]:
grouped = df[df['season']=='Summer'].groupby(['year', 'athlete_id']).agg(athlete_cnt=('athlete_id', 'count')).reset_index()
gr2 = grouped[grouped['athlete_cnt']==1].groupby(['year']).agg(cnt_one=('athlete_id', 'count')).reset_index()
gr2.head(2)

,year,cnt_one
0,1896,104
1,1900,1048


In [53]:
gr3 = grouped[grouped['athlete_cnt']>1].groupby(['year']).agg(cnt_many=('athlete_id', 'count')).reset_index()
grouped = pd.merge(gr2, gr3, on=['year'])
grouped.head(2)

,year,cnt_one,cnt_many
0,1896,104,139
1,1900,1048,634


In [54]:
fig = go.Figure()
fig.add_bar(x=grouped['year'], y=grouped['cnt_one'], 
            marker=dict(color='#34aeeb', opacity=0.8), width=0.7, name='Only one')
fig.add_bar(x=grouped['year'], y=grouped['cnt_many'],
            marker=dict(color='#eb09d8', opacity=0.8), width=0.3, name='More than one')
fig.update_layout(barmode='overlay', height=450, width=950, xaxis_title='Year', yaxis_title='Athletes',
title="Number of athletes who participated only in one event and <br>Number of athletes who participated at least in one event during the Olympics",
)
fig.update_xaxes(type = 'category', dtick=2)
fig.show()

Note: This graph covers only the Summer Olympics. <br>

Surprisingly, the number of athletes who participate in more than one event doesn't proportionally increase <br> with those who participate only in one event.

In [55]:
gr1 = df[(df['event'].str.contains('Women')) & (df['season']=='Summer')].\
    groupby('year').agg({'event': 'nunique'}).rename({'event': 'woman_cnt'}, axis=1).reset_index()
gr2 = df[(df['event'].str.contains('Men')) & (df['season']=='Summer')].\
    groupby('year').agg({'event': 'nunique'}).rename({'event': 'man_cnt'}, axis=1).reset_index()

In [56]:
df_woman = pd.merge(gr1, gr2, on='year')
df_woman['woman proportion'] = (df_woman['woman_cnt']*100/(df_woman['woman_cnt']+df_woman['man_cnt'])).round(1)
df_woman.head(2)

,year,woman_cnt,man_cnt,woman proportion
0,1900,3,132,2.2
1,1904,4,140,2.8


In [57]:
w_ratio =df[df['season']=='Summer'].groupby(['year', 'sex']).agg({'athlete_id': 'nunique'}).rename({'athlete_id': 'cnt'}, axis=1).unstack()
w_ratio = w_ratio.droplevel(level=0, axis=1)
w_ratio = w_ratio.reset_index().rename_axis(None, axis=1).dropna(subset=['Female', 'Male'], how='any')
w_ratio['w_proportion'] = (w_ratio['Female']*100/(w_ratio['Female']+w_ratio['Male'])).round(1)
w_ratio.head()

,year,Female,Male,w_proportion
0,1896,2,241,0.8
1,1900,30,1652,1.8
2,1904,6,1361,0.4
3,1908,54,2358,2.2
4,1912,62,2813,2.2


In [58]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("Share of women's events only", "Share of women's athletes"))
fig.add_scatter(x=df_woman['year'], y=df_woman['woman proportion'], mode='lines', 
                showlegend=False, fill='tozeroy', fillcolor='rgba(235, 87, 166, 0.4)', line_color='#eb57a6', 
                hoverinfo='x+y', row=1, col=1)
fig.add_scatter(x=w_ratio['year'], y=w_ratio['w_proportion'],
                showlegend=False, fill='tozeroy', fillcolor='rgba(235, 87, 166, 0.4)', line_color='#eb57a6',
                hoverinfo='x+y', row=1, col=2)

fig.update_layout(yaxis1_ticksuffix='%', xaxis1_title='Year', yaxis2_ticksuffix='%', xaxis2_title='Year',
                   title='Female side of Olympics (only the Summer ones)', title_x=0.5, title_y=0.96)
fig.update_annotations(yshift=15)
fig.show()

We see a good correlation between the increase in the number of women's events and women's athletes over time. <br>
At first there were almost no women in sports but now there is almost 50% of women in the Olympic sports.

In [59]:
ages = df[(df['medal'].notna()) & (df['year']>1930) & (df['season']=='Summer')].\
    groupby(['year', 'age_category'], observed=True)['medal'].count().reset_index()
ages2 = ages.groupby('year')['medal'].sum().reset_index().rename({'medal': 'total_cnt'}, axis=1)
ages = ages.merge(ages2, on='year')
ages['percent'] = (ages['medal']*100/ages['total_cnt']).round(1)

In [60]:
fig = px.bar(ages, x='age_category', y='medal', height=500, width=500, text=[f"{percent}%" for percent in ages['percent']],
             animation_frame='year', range_y=[0,1200])
fig.update_layout(xaxis_title='Age group', yaxis_title = 'Medals', title='Number of medals earned in each age group', title_x=0.5)
fig.update_traces(textposition = "outside")
for frame in fig.frames:
    for trace in frame.data:
        trace.textposition = 'outside'
fig.show()

Starting from 1932 and considering only Summer Olympics. <br>
We can see that the percentages of "26-30" and "31-35" group cateories increase over time which can be explained by the fact that <br>
people remain physically active in their 30-s, plus human lifespan has increased  in the last 100 years.

In [61]:
gr1 = df[((df['year']==1988) | (df['year']==1996)) & (df['season']=='Summer')].groupby(['year', 'country', 'sport', 'event', 'medal']).\
    agg({'medal': 'count'}).rename(columns={"medal": 'cnt'}).reset_index()
gr1.head(3)

,year,country,sport,event,medal,cnt
0,1988,Argentina,Tennis,"Singles, Women",Silver,1
1,1988,Argentina,Volleyball,"Volleyball, Men",Bronze,12
2,1988,Australia,Athletics,"400 metres Hurdles, Women",Gold,1


In [62]:
gr1['cnt']= 1  # to make team medals count as one.

In [63]:
gr1 = gr1.groupby(['year', 'country', 'medal']).agg({'cnt': 'sum'}).sort_values(by='cnt', ascending=False).reset_index()
gr2 = gr1.pivot_table(index=['year', 'country'], columns='medal', values='cnt', fill_value=0, margins=True, aggfunc='sum').reset_index()
gr2.head()

medal,year,country,Bronze,Gold,Silver,All
0,1988,Argentina,1,0,1,2
1,1988,Australia,5,3,6,14
2,1988,Austria,0,1,0,1
3,1988,Belgium,2,0,0,2
4,1988,Brazil,3,1,2,6


In [64]:
gr2.sort_values(by=['year', 'All', 'Gold', 'Silver'], ascending=[True, False, False, False], inplace=True)
gr2 = gr2.drop(gr2[gr2['year']=='All'].index, axis=0)
gr2.reset_index(drop=True, inplace=True)
gr2.rename_axis(None, axis=1, inplace=True)
gr2.head()

,year,country,Bronze,Gold,Silver,All
0,1988,Soviet Union,46,54,31,131
1,1988,East Germany,30,37,35,102
2,1988,United States,27,36,31,94
3,1988,West Germany,15,11,14,40
4,1988,Bulgaria,13,10,12,35


In [65]:
gr3 = gr1[(gr1['year']==1988) & (gr1['country'].isin(gr2[gr2['year']==1988][:20]['country']))]
gr4 = gr1[(gr1['year']==1996) & (gr1['country'].isin(gr2[gr2['year']==1996][:20]['country']))]

In [66]:
fig = px.bar(gr3, x='cnt', y='country', color='medal', barmode='stack',
             category_orders={"medal": ["Gold", "Silver", "Bronze"]},
             color_discrete_map = {'Gold': '#FFD700', 'Silver': '#b8b5b0', 'Bronze': '#cd7f32'})
fig.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': gr2[gr2['year']==1988][:20][::-1]['country']},
                  xaxis_title='Medals', yaxis_title='Country', title='Total number of medals per country in Summer Olympics 1988',
                  title_x=0.5)  
for i in range(20):
    fig.add_annotation(x=gr2[gr2['year']==1988].reset_index(drop=True)['All'][i]+1.9,  # 1.7
                       y=gr2[gr2['year']==1988].reset_index(drop=True)['country'][i],
                       text=str(gr2[gr2['year']==1988].reset_index(drop=True)['All'][i]),
                       showarrow=False)
fig.show()
#######################################################################
fig2 = px.bar(gr4, x='cnt', y='country', color='medal', barmode='stack', 
             color_discrete_sequence=['#FFD700', '#b8b5b0', '#cd7f32'])
fig2.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': gr2[gr2['year']==1996][:20][::-1]['country']},
                  xaxis_title='Medals', yaxis_title='Country', title='Total number of medals per country in Summer Olympics 1996',
                  title_x=0.5)  
for i in range(20):
    fig2.add_annotation(x=gr2[gr2['year']==1996]['All'].reset_index(drop=True)[i]+1.7,  
                       y=gr2[gr2['year']==1996]['country'].reset_index(drop=True)[i],
                       text=str(gr2[gr2['year']==1996]['All'].reset_index(drop=True)[i]),
                       showarrow=False)
fig2.show()

Between these dates two significant events took place, namely a reunification of Germany in 1990 and collapse of Soviet Union in 1991. <br>
Comparing the Summer 1988 and 1996 Olympics we can observe the following: <br>

1. Soviet Union broke into 15 independet countires which made Russia lose many medals. <br>
The biggest chunk got taken by Ukraine with 23 medals, the second biggest part got taken by Belarus with 15 medals. <br>
2. West and East Germany got reunited but somehow earned less medals (142 in total) in 1996 <br> than separately before that in 1988 with 65 total medals.

### The summary <br>

1. The number of athletes, events and countries in the olympic games has been generally increasing over time. <br>
The spikes can be usually explained by understanding political and economical situations that took place in the respective year. <br>
2. The Summer Olympics has more participants, event than the Winter. The Summer Olympics is more "important" and draws more attention. <br>
3. The number of athletes who participate in more than only event stays relatively the same compared to those who participate <br>
in only one event.
4. At first there was almost no women in the olympics. Hoever, in the last years 2020, 2022 woman's ratio has reached almost 50%. <br>
5. Age group from 18 to 25 produces the most of medals earned, the second most fruitful group is 26-30 which is catching up with the first one. <br>
6. in 1996 Soviet Union (Russia) lost many medals, because it split into different independent countries. However, Germany's number <br>
of medals also decreased even though the country reunited into one country (East + West Germany).